In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import VectorDBQA
import sentence_transformers
import pandas as pd

/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13153304
      0 drwxr-xr-x   17 kariato  staff   544B Feb 21 00:04 ./
      0 drwxr-xr-x    7 kariato  staff   224B Feb 10 21:29 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Feb 10 14:58 .DS_Store
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 21 00:04 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final (1).pdf
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 20 10:15 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final.pdf
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x    4 kariato  staff   128B Feb 13 22:03 chroma2_db/
      0 drwxr-xr-x    4 kariato  staff   128B Feb 12 02:29 chroma_db/
2785400 -rw-rw-r--@   1 kariato  staff   1.3G Sep 19  2019 emails.csv
2099696 -rw-r--r--    1 kariato  staff   1.0G Feb 10 14:24 enron.sql
 427936 -rw-r--r--@   1 kariato  staff   209M Feb 

In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Volumes/External/opt/anaconda3/envs/ollama/lib/python3.10/site-packages/torch/_utils.py:841: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
db = Chroma(embedding_function=embedding_function,persist_directory="./chroma_db")
db2 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma2_db")
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)

In [5]:
import re
def cleanText(text):
        
    text = text.replace('\\n','')
    text = text.replace('\\','')
    #text = text.replace('\t', '')
    #text = re.sub('\[(.*?)\]','',text) #removes [this one]
    #text = re.sub('(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?\s',
    #            ' __url__ ',text) #remove urls
    #text = re.sub('\'','',text)
    #text = re.sub(r'\d+', ' __number__ ', text) #replaces numbers
    #text = re.sub('\W', ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [19]:
query = "mark to market accounting offsetting concerns hiding losses"
docs = db2.similarity_search_with_score(query,k=400)
topics=[]
for doc in docs:
    details = cleanText(doc[0].page_content)
    print(doc[1])
    if details not in topics:
        topics.append(details)
text_for_topics = " ".join(topics)
#topics

0.7306435108184814
0.7306435108184814
0.731531023979187
0.731531023979187
0.731531023979187
0.7477731108665466
0.7477731108665466
0.7477731108665466
0.7477731108665466
0.7607764601707458
0.7607764601707458
0.7607764601707458
0.7607764601707458
0.7694604396820068
0.7694604396820068
0.7694604396820068
0.7694604396820068
0.8205325603485107
0.8205325603485107
0.8205325603485107
0.8205325603485107
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8731479644775391
0.8733437061309814
0.8733437061309814
0.8733437061309814
0.8733437061309814
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8737673759460449
0.8776491284370422
0.8776491284370422
0.8776491284370422
0.8776491284370422
0.8776491284370422
0.8776491284370422
0.8776491284370422
0.8776491284370

In [20]:
from bertopic import BERTopic
model = BERTopic(nr_topics="auto")
cluster_topics, probas = model.fit_transform(topics)

In [21]:
freq = model.get_topic_info()
print("Number of topics: {}".format( len(freq)))
freq

Number of topics: 3


,Topic,Count,Name,Representation,Representative_Docs
0,-1,17,-1_the_to_mc_in,"[the, to, mc, in, we, and, on, of, bpa, that]",[VinceRoger G Leworthy@ENRON_DEVELOPMENT04/10/...
1,0,39,0_the_to_of_and,"[the, to, of, and, in, that, for, on, accounti...","[""There could be a quality-of-earnings issue,""..."
2,1,34,1_the_to_of_and,"[the, to, of, and, on, for, is, in, that, this]",[Subject: RE: MTM IncomeGOOD JOB!!!!! -----Ori...


In [27]:
query = "sbe and raptor concerns about hiding loses"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

ssue even more stock to keep Raptor afloat.=20
THE WRITE-DOWN As of last week, these transactions were not accounted for o=
n Enron's books. What this means is that Enron may have to restate its earn=
ings yet again, posting even bigger losses.=20
Quote: Skilling's abrupt departure will raise suspicions of accounting impr=
oprieties and valuation issues... --Excerpts from the letter Watkins sent t=
o Lay on Aug. 15, 2001 I am incredibly nervous that we will implode in a wa=
ve of accounting scandals... It sure looks to the layman on the street that=
 we are hiding losses in a related company...
ssue even more stock to keep Raptor afloat.=20
THE WRITE-DOWN As of last week, these transactions were not accounted for o=
n Enron's books. What this means is that Enron may have to restate its earn=
ings yet again, posting even bigger losses.=20
Quote: Skilling's abrupt departure will raise suspicions of accounting impr=
oprieties and valuation issues... --Excerpts from the letter Watkins sent

In [28]:
query = "accounting aggresive hiding loses"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

Subject: 

T
he first detailed allegations that Enron
abused mark-to-market accounting to
hide wholesale trading losses have
emerged in congressional testimony pro-vided
by a professor and former deriva-tives
trader who reviewed the Enron case
for Senate Governmental Affairs Com-mittee.
Those charges, along with allega-tions
that have surfaced that mark-to-market
accounting was misused by Enron
Energy Services, the company's retail
energy and services unit, are expected to
support the push by accounting rulemak-ers
for much more disclosure on mark-to-market
accounting practices and the
impact of mark-to-market accounting on
earnings trading firms' earnings.
In testimony Thursday before the
Senate Governmental Affairs Committee
last week, Frank Partnoy of the Universi-ty
of San Diego Law School, said it
appears that Enron traders mismarked
forward curves and manipulated the
amount of profits and losses that would
be reported in financial statements.
"In a nutshell, it appears that some


In [29]:
query = "mark to market hiding loses in rapotor"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

Subject: Re: Enron Raptor I P&L Reversal

The change to the DPR will occur on the 14th, not on the 13th as previously 
communicated.

Best regards

Shona Wilson
X39123


   
	
	
	

Maria LeBeau/HOU/ECT@ECT, L'Sheryl Hudson/HOU/ECT@ECT, Ted 
Murphy/HOU/ECT@ECT, David Port/Market Risk/Corp/Enron@ENRON 

Subject: Enron Raptor I P&L Reversal

 On the 11/13/00 MPR, we will be offsetting $163,337,947 of YTD P&L loss 
associated with the Raptor I hedge.  This P&L has been accumulating since 
Project Raptor was created using 8/3/00 asset balances.  On the MPR, we have 
been showing the daily marking of the Raptor assets to the screen but have 
not been showing the corresponding hedge offset.  The amounts have been 
flashed by ENA with the hedge offset included.

 This change will show up on the 11/13/00 DPR.

 If you have any questions regarding this matter, please contact either Susie 
Ayala at x3-0524 or myself at x3-6983 by 10:00 a.m. on Tuesday, 11/14/00.

 Thank you,
 David Maxwell
Subjec

In [33]:
query = "california market manipulation"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

Subject: FERC Staff market mitigation comments for California--Comments of
 Enron--DUE WEDNESDAY

I forgot to copy you ...
---------------------- Forwarded by Alan Comnes/PDX/ECT on 03/20/2001 10:00 
PM ---------------------------


Alan Comnes
03/20/2001 09:54 PM
Richter/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Steve C Hall/PDX/ECT@ECT, 
Bill Williams III/PDX/ECT@ECT

Subject: FERC Staff market mitigation comments for California--Comments of 
Enron--DUE WEDNESDAY

To West Desk Traders in Cali:

On Thursday 3/22, FERC will accept comments on its staff's plan to provide 
market mitigation to California ISO markets.  A summary of the report is 
copied below.

Enron GA plans to file a motion that will support the comments of EPSA and 
WPTF and supplement the comments as noted below. 

EPSA's comments are generally supportive of the staff report but recommends 
that FERC:

clarify that staff's market mitigation measures are being put into place to 
address structural defects in the Californ